# YOLO 기본 객체 탐지 실험

이 노트북에서는 YOLO 모델을 사용하여 기본적인 객체 탐지를 수행합니다.

## 목차
1. 환경 설정 및 라이브러리 임포트
2. 설정 및 모델 초기화
3. 샘플 이미지 다운로드
4. 단일 이미지 탐지
5. 결과 시각화
6. 결과 저장

## 1. 환경 설정 및 라이브러리 임포트

In [ ]:
import sys
from pathlib import Path

# 프로젝트 루트를 파이썬 경로에 추가
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# 라이브러리 임포트
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch

# 프로젝트 모듈 임포트
from src.config import YOLOConfig, PRESET_CONFIGS
from src.data.loader import DataLoader
from src.model.detector import YOLODetector
from src.output.visualizer import Visualizer
from src.output.exporter import ResultExporter

print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA 디바이스: {torch.cuda.get_device_name(0)}")

## 2. 설정 및 모델 초기화

### 사전 정의된 설정 사용
- `fast`: YOLOv8n (nano) - 빠른 속도
- `balanced`: YOLOv8s (small) - 속도와 정확도 균형
- `accurate`: YOLOv8m (medium) - 높은 정확도

In [ ]:
# 설정 선택 (fast, balanced, accurate)
config = PRESET_CONFIGS["fast"]
print(config)

In [ ]:
# 또는 커스텀 설정 사용
# config = YOLOConfig(
#     model_name="yolov8n",
#     conf_threshold=0.25,
#     iou_threshold=0.45,
# )

In [ ]:
# YOLO 모델 초기화
detector = YOLODetector(config)

# 모델 정보 확인
model_info = detector.get_model_info()
print(f"\n모델 정보:")
print(f"  - 모델: {model_info['model_name']}")
print(f"  - 디바이스: {model_info['device']}")
print(f"  - 클래스 수: {model_info['num_classes']}")
print(f"  - 신뢰도 임계값: {model_info['conf_threshold']}")

## 3. 샘플 이미지 다운로드

COCO 데이터셋 샘플 이미지를 다운로드합니다.

In [ ]:
import urllib.request

# 샘플 이미지 URL (COCO 데이터셋)
sample_urls = [
    "https://raw.githubusercontent.com/ultralytics/yolov5/master/data/images/bus.jpg",
    "https://raw.githubusercontent.com/ultralytics/yolov5/master/data/images/zidane.jpg",
]

# 샘플 디렉토리 생성
sample_dir = project_root / "data" / "samples"
sample_dir.mkdir(parents=True, exist_ok=True)

# 이미지 다운로드
sample_paths = []
for url in sample_urls:
    filename = sample_dir / Path(url).name
    if not filename.exists():
        print(f"다운로드 중: {url}")
        urllib.request.urlretrieve(url, filename)
    sample_paths.append(filename)
    print(f"저장됨: {filename}")

print(f"\n총 {len(sample_paths)}개 이미지 준비 완료")

## 4. 단일 이미지 탐지

In [ ]:
# 데이터 로더 초기화
data_loader = DataLoader(use_cv2=True)

# 이미지 로딩
image_path = sample_paths[0]
image = data_loader.load_image(image_path)

# 이미지 정보
image_info = data_loader.get_image_info(image)
print(f"이미지 정보:")
print(f"  - 경로: {image_path.name}")
print(f"  - 크기: {image_info['width']}x{image_info['height']}")
print(f"  - 채널: {image_info['channels']}")

In [ ]:
# 객체 탐지 수행
print("\n객체 탐지 수행 중...")
detections = detector.predict(image, verbose=True)

print(f"\n탐지된 객체 수: {len(detections)}")
print("\n탐지 결과:")
for i, det in enumerate(detections, 1):
    print(f"{i}. {det}")

## 5. 결과 시각화

In [ ]:
# 시각화 도구 초기화
visualizer = Visualizer(
    line_thickness=2,
    font_scale=0.5,
    hide_labels=False,
    hide_conf=False,
)

# 탐지 결과 시각화
visualizer.show(
    image,
    detections,
    figsize=(12, 8),
    title=f"Detection Results: {image_path.name}",
)

In [ ]:
# 탐지 결과 요약 출력
visualizer.print_detection_summary(detections)

## 6. 결과 저장

In [ ]:
# 결과 저장 도구 초기화
exporter = ResultExporter(output_dir=project_root / "data" / "results")

# 탐지 결과가 그려진 이미지 생성
annotated_image = visualizer.draw_detections(image, detections)

# 이미지 저장
image_output_path = exporter.save_image(
    annotated_image,
    filename=f"detected_{image_path.name}",
)

# JSON 저장
json_output_path = exporter.save_detections_json(
    detections,
    filename=f"detections_{image_path.stem}.json",
    metadata={
        "source_image": str(image_path),
        "model": config.model_name,
        "conf_threshold": config.conf_threshold,
    },
)

print(f"\n결과 저장 완료:")
print(f"  - 이미지: {image_output_path}")
print(f"  - JSON: {json_output_path}")

## 추가 실험: 배치 처리

여러 이미지를 한 번에 처리합니다.

In [ ]:
# 모든 샘플 이미지 로딩
images = data_loader.load_batch(sample_paths)
print(f"로딩된 이미지: {len(images)}개")

# 배치 탐지
print("\n배치 탐지 수행 중...")
batch_detections = detector.predict_batch(images, verbose=False)

# 결과 요약
total_objects = sum(len(dets) for dets in batch_detections)
print(f"\n총 탐지된 객체 수: {total_objects}")
for i, (path, dets) in enumerate(zip(sample_paths, batch_detections), 1):
    print(f"{i}. {path.name}: {len(dets)}개 객체")

In [ ]:
# 배치 결과 시각화 (비교)
annotated_images = [
    visualizer.draw_detections(img, dets)
    for img, dets in zip(images, batch_detections)
]

titles = [f"{path.name}\n{len(dets)} objects" for path, dets in zip(sample_paths, batch_detections)]

fig = visualizer.create_comparison(
    annotated_images,
    batch_detections,
    titles=titles,
    figsize=(15, 5),
)
plt.show()

In [ ]:
# 배치 결과 저장
batch_results = exporter.save_batch_results(
    annotated_images,
    batch_detections,
    image_names=[p.name for p in sample_paths],
    save_images=True,
    save_json=True,
)

print(f"\n배치 결과 저장 완료:")
print(f"  - 저장된 이미지: {len(batch_results['images'])}개")
print(f"  - 저장된 JSON: {len(batch_results['json'])}개")
print(f"  - 총 탐지 객체: {batch_results['summary']['total_detections']}개")

In [ ]:
# 요약 보고서 생성
report_path = exporter.create_summary_report(
    batch_detections,
    filename="detection_summary.txt",
)

print(f"\n요약 보고서: {report_path}")

# 보고서 내용 출력
with open(report_path, "r", encoding="utf-8") as f:
    print(f.read())

## 결론

이 노트북에서 다룬 내용:
1. ✅ YOLO 모델 초기화 및 설정
2. ✅ 단일 이미지 객체 탐지
3. ✅ 탐지 결과 시각화
4. ✅ 결과 저장 (이미지, JSON)
5. ✅ 배치 처리 및 비교 시각화
6. ✅ 요약 보고서 생성

### 다음 단계
- 02_batch_processing.ipynb: 대량 이미지 처리
- 03_performance_analysis.ipynb: 모델 성능 분석 및 최적화